In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
from sklearn.metrics import accuracy_score
import operator
from timeit import default_timer as timer
from src.main import prepare_attribut_data, prepare_result_data
from src.main import convert_to_one_hot


In [4]:

train_path_away = '../data/Train_Data/train_away_team_statistics_df.csv'
train_path_home = '../data/Train_Data/train_home_team_statistics_df.csv'
train_path_result = '../data/Train_Data/Y_train.csv'
df_attribut = prepare_attribut_data(train_path_away, train_path_home)
df_result = prepare_result_data(train_path_result)

In [ ]:
df_attribut.head()

In [ ]:
df_result.head()

In [ ]:
from sklearn.model_selection import train_test_split

attribut_train, attribut_test, result_train, result_test = train_test_split(df_attribut, df_result, test_size=0.001,
                                                                            random_state=10)


In [ ]:
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import LogisticRegression

max_iter_list = [3000]
solver_list = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky']  # sag and saga take a lot of time
regularization_factor_list = [1, 0.1, 0.01, 0.001, 0.0001]
accuracy_dict = {}
time_dict = {}
LR_dict = {}
for max_iter in max_iter_list:
    for solver in solver_list:
        for regularization_factor in regularization_factor_list:
            start = timer()

            LR = LogisticRegression(max_iter=max_iter, solver=solver, C=regularization_factor).fit(attribut_train,
                                                                                                   result_train)
            yhat = LR.predict(attribut_test)
            accuracy = np.round(accuracy_score(result_test, yhat), 4)
            key = f'solver_{solver}_iter_{max_iter}_C_{regularization_factor}'
            accuracy_dict[key] = accuracy
            LR_dict[key] = LR

            end = timer()
            print(f'for this hyperparam {key} the training took ')

            delta = end - start
            minutes = round(delta // 60)
            delta %= 60
            seconds = round(delta)
            time_str = f"{minutes:02d}m{seconds:02d}s"
            time_dict[key] = time_str
            print(time_str)



In [ ]:
accuracy_dict

In [ ]:
max_accuracy = max(accuracy_dict.items(), key=operator.itemgetter(1))
LR_best = LR_dict[max_accuracy[0]]

In [ ]:
time_dict

In [ ]:
max_accuracy

In [ ]:
yhat = LR_best.predict(attribut_test)
np.round(accuracy_score(result_test, yhat), 4)

In [ ]:
yhat

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(result_test, yhat)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=[-1, 0, 1], )
disp.plot(cmap="Blues")

In [5]:
test_path_away = '../data/Test_Data/train_away_team_statistics_df.csv'
test_path_home = '../data/Test_Data/train_home_team_statistics_df.csv'
df_attribut_test = prepare_attribut_data(test_path_away, test_path_home)

FileNotFoundError: [Errno 2] No such file or directory: '../data/Test_Data/train_away_team_statistics_df.csv'

In [ ]:
df_attribut_test

In [ ]:
yhat_test = LR_best.predict(df_attribut_test)

In [ ]:
y_pred_test = pd.DataFrame(yhat_test)
y_pred_test = convert_to_one_hot(yhat_test)

In [ ]:
y_pred_test

In [ ]:
test_prediction = pd.concat([df_attribut_test['ID'], y_pred_test], axis=1)

In [ ]:
test_prediction.to_csv('./submission/logistic_benchmark_submission.csv', index=False)